In [ ]:
# import os
# import pandas as pd
# from langchain_community.document_loaders import CSVLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import OllamaEmbeddings

# FACTS_CSV = "facts.csv"   # CSV kept outside (same folder as app.py)
# DB_DIR = "vectorstore"


# ## Create data folder
# os.makedirs(DATA_DIR, exist_ok=True)
# os.makedirs(DB_DIR, exist_ok=True)



Checking CSV at: data/facts.csv
✅ CSV Loaded Successfully!
                                                text      source        date
0  Government announces PM-KISAN scheme to provid...  pib.gov.in  2019-02-24
1  The Union Cabinet approves National Education ...  pib.gov.in  2020-07-29
2  The Ministry of Power launches the Saubhagya s...  pib.gov.in  2017-09-25
3  Digital India initiative expands broadband acc...  pib.gov.in  2021-03-15
4  Ayushman Bharat provides free health insurance...  pib.gov.in  2018-09-23
Columns: Index(['text', 'source', 'date'], dtype='object')
Using source_column: text
🚀 Vectorstore created successfully!


In [ ]:
# # Step 1: check CSV
# print("Checking CSV at:", FACTS_CSV)
# df = pd.read_csv(FACTS_CSV)
# print("CSV Loaded Successfully!")
# print(df.head())
# print("Columns:", df.columns)

# # 🔍 Step 2: pick correct column (first one)
# SOURCE_COL = df.columns[0]      # e.g. 'statement', 'fact', etc.
# print("Using source_column:", SOURCE_COL)

# # Build vectorstore from CSV
# loader = CSVLoader(file_path=csv_path, source_column=df.columns[0])
# docs = loader.load()

# splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
# chunks = splitter.split_documents(docs)

# embed = OllamaEmbeddings(model=EMBED_MODEL)
# vectorstore = Chroma.from_documents(chunks, embed, persist_directory=DB_DIR)
# vectorstore.persist()

# print("Vectorstore created successfully!")


Checking CSV at: data/facts.csv
CSV Loaded Successfully!
                                                text      source        date
0  Government announces PM-KISAN scheme to provid...  pib.gov.in  2019-02-24
1  The Union Cabinet approves National Education ...  pib.gov.in  2020-07-29
2  The Ministry of Power launches the Saubhagya s...  pib.gov.in  2017-09-25
3  Digital India initiative expands broadband acc...  pib.gov.in  2021-03-15
4  Ayushman Bharat provides free health insurance...  pib.gov.in  2018-09-23
Columns: Index(['text', 'source', 'date'], dtype='object')
Using source_column: text
Vectorstore created successfully!


In [ ]:
# # load vectorstore and retriever
# embed = OllamaEmbeddings(model=EMBED_MODEL)
# vectorstore = Chroma(persist_directory=DB_DIR, embedding_function=embed)
# retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})

# llm = Ollama(model=LLM_MODEL)

# query = "The Indian government has announced free electricity to all farmers starting July 2025."

# docs = retriever.get_relevant_documents(query)
# context = "\n\n".join([d.page_content for d in docs]) if docs else ""

# prompt = (
#     "You are a fact-checking assistant.\n\n"
#     "Using ONLY the evidence below, evaluate the user's question and return EXACTLY one JSON object with keys: verdict, evidence, reasoning.\n\n"
#     "Evidence:\n" + context + "\n\n"
#     "Question: " + query + "\n\n"
#     "Return only the JSON object (no additional commentary)."
# )

# resp = llm.invoke(prompt)
# raw = resp.content if hasattr(resp, "content") else str(resp)
# print("Raw output:", raw)

# # try to parse
# import re, json
# m = re.search(r"\{.*\}", raw, flags=re.S)
# if m:
#     try:
#         parsed = json.loads(m.group(0))
#         print("Parsed JSON:", json.dumps(parsed, indent=2))
#     except Exception as e:
#         print("Failed to parse JSON:", e)
# else:
#     print("No JSON found in output.")


Raw output: ```json
{
  "verdict": "Cannot be evaluated",
  "evidence": null,
  "reasoning": "The provided evidence focuses on India achieving 100% village electrification under a program called Deen Dayal Upadhyaya Gram Jyoti Yojana. There is no information about the Indian government announcing free electricity to all farmers."
}
```
Parsed JSON: {
  "verdict": "Cannot be evaluated",
  "evidence": null,
  "reasoning": "The provided evidence focuses on India achieving 100% village electrification under a program called Deen Dayal Upadhyaya Gram Jyoti Yojana. There is no information about the Indian government announcing free electricity to all farmers."
}


In [ ]:
import os
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
import json, re

# ---------------- CONFIG ----------------
FACTS_CSV = "facts.csv"            # your CSV must be next to the notebook
DB_DIR = "vectorstore"             # where Chroma DB will be saved
EMBED_MODEL = "nomic-embed-text"   # embedding model
LLM_MODEL = "gemma2:2b"            # LLM model for checking
TOP_K = 3
# ----------------------------------------

# Create DB directory
os.makedirs(DB_DIR, exist_ok=True)

# Step 1: Load CSV
print("Checking CSV at:", FACTS_CSV)
df = pd.read_csv(FACTS_CSV)
print("CSV Loaded Successfully!")
print(df.head())
print("Columns:", df.columns)

# Automatically pick the first column
SOURCE_COL = df.columns[0]
print("Using source_column:", SOURCE_COL)

# Step 2: Build vectorstore
loader = CSVLoader(file_path=FACTS_CSV, source_column=SOURCE_COL)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embed = OllamaEmbeddings(model=EMBED_MODEL)
vectorstore = Chroma.from_documents(chunks, embed, persist_directory=DB_DIR)
vectorstore.persist()

print("Vectorstore created successfully!")

# Step 3: Load DB & Create retriever
embed = OllamaEmbeddings(model=EMBED_MODEL)
vectorstore = Chroma(persist_directory=DB_DIR, embedding_function=embed)
retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})

llm = Ollama(model=LLM_MODEL)

# Step 4: Test Query
query = "The Indian government has announced free electricity to all farmers starting July 2025."

docs = retriever.get_relevant_documents(query)
context = "\n\n".join([d.page_content for d in docs]) if docs else ""

prompt = (
    "You are a fact-checking assistant.\n\n"
    "Using ONLY the evidence below, evaluate the user's question and return EXACTLY one JSON object with keys: verdict, evidence, reasoning.\n\n"
    "Evidence:\n" + context + "\n\n"
    "Question: " + query + "\n\n"
    "Return only the JSON object."
)

resp = llm.invoke(prompt)
raw = resp.content if hasattr(resp, "content") else str(resp)
print("Raw output:\n", raw)

# Step 5: Parse JSON
m = re.search(r"\{.*\}", raw, flags=re.S)
if m:
    try:
        parsed = json.loads(m.group(0))
        print("\nParsed JSON:")
        print(json.dumps(parsed, indent=2))
    except Exception as e:
        print("Failed to parse JSON:", e)
else:
    print("No JSON found in output.")
